# <center> Финальное задание </center>
## <center> Предсказание пола клиента по транзакциям</center>

## Описание задачи
#### Ваше задание - предсказать пол клиента, основываясь на его транзакционных исторических данных. Выполнение финального задания - это маленький шаг в большую Data Science-всесенную, поэтому отнеситесь к нему максимально серьёзно :)
#### В роли метрики выступает [ROC AUC](https://dyakonov.org/2017/07/28/auc-roc-%D0%BF%D0%BB%D0%BE%D1%89%D0%B0%D0%B4%D1%8C-%D0%BF%D0%BE%D0%B4-%D0%BA%D1%80%D0%B8%D0%B2%D0%BE%D0%B9-%D0%BE%D1%88%D0%B8%D0%B1%D0%BE%D0%BA/), который и нужно будет оптимизировать.

In [2]:
!pip install xgboost

In [3]:
!pip install joblib

In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
import re
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import joblib

from tqdm._tqdm_notebook import tqdm_notebook

<ipython-input-4-dd84166dfa05>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [15]:
PATH_DATA = ''
MODEL_PATH = "model.pkl"

In [16]:
# Считываем данные
tr_mcc_codes = pd.read_csv(os.path.join(PATH_DATA, 'mcc_codes.csv'), sep=';', index_col='mcc_code')
tr_types = pd.read_csv(os.path.join(PATH_DATA, 'trans_types.csv'), sep=';', index_col='trans_type')

transactions = pd.read_csv(os.path.join(PATH_DATA, 'transactions.csv'), index_col='client_id')
gender_train = pd.read_csv(os.path.join(PATH_DATA, 'train.csv'), index_col='client_id')
gender_test = pd.read_csv(os.path.join(PATH_DATA, 'test.csv'), index_col='client_id')
transactions_train = transactions.join(gender_train, how='inner')
transactions_test = transactions.join(gender_test, how='inner')

# My Things

In [17]:
transactions.head()

,trans_time,mcc_code,trans_type,amount,term_id,trans_city
client_id,,,,,,
d1bbbc9a0e0410d3cf12a3d2f44f3450,35 08:24:41,4829.0,2370.0,-1808.56,NaN,Tver
81c7349f7bdcb661bc7d5f5f0fdac6aa,351 13:27:15,6011.0,7010.0,7954.51,442207,Kazan
fc0c7c937f139bd6d1c5b5d0a0023dd3,150 16:03:59,6011.0,2010.0,-12293.56,NaN,Khabarovsk
3495487d5beaa5caaa0682fddc38379a,373 11:54:12,6010.0,7070.0,49.52,888987,Kaliningrad
07e279042b1da23710fa2f65034d30b9,340 13:11:45,6010.0,7070.0,45.29,888918,Saint Petersburg


In [18]:
transactions.tail()

,trans_time,mcc_code,trans_type,amount,term_id,trans_city
client_id,,,,,,
9882b95ebce98b7972c4eed69e125ac5,224 15:24:31,6011.0,2010.0,-722.15,786537,Vladivostok
44c46e9eafc561d6eba535b2fb33321d,292 13:09:35,6011.0,2011.0,-1229.06,620932,Moscow
8d69e4ecae4b509d528b9a0168f749d0,451 14:56:40,6010.0,7070.0,268.36,888910,Kazan
36ca5021ca85333e815bda3e08f1ad34,225 06:12:08,4814.0,1030.0,-216.54,888995,Penza
0238bd70b4540076e5,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
transactions.describe()

,mcc_code,trans_type,amount
count,116855.000000,116855.000000,1.168550e+05
mean,5587.078439,2449.149604,-6.814339e+02
std,602.161694,2221.955923,1.618977e+04
min,742.000000,1010.000000,-1.084683e+06
25%,5211.000000,1030.000000,-7.223200e+02
50%,5541.000000,1110.000000,-1.840300e+02
75%,6010.000000,2370.000000,-3.701000e+01
max,9402.000000,8100.000000,2.169367e+06
